In [1]:
import ollama
import os

import numpy as np


In [2]:
def embed(doc):
    return ollama.embed(model="nomic-embed-text", input=doc)['embeddings']
    #model="avr/sfr-embedding-mistral"
#vector_store.add_documents(documents=chunks, ids=uuids)

In [3]:
#use sklearn
from sklearnex import patch_sklearn
patch_sklearn()

from  sklearn.neighbors import NearestNeighbors

short_search = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='auto', metric='cosine')

long_search = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='auto', metric='cosine')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
import joblib
import h5py
import json
from tqdm import tqdm
long_chunks = joblib.load('chunks.pkl')

long_chunks_embed = np.zeros((len(long_chunks), 768))
long_strings =[]
for i in tqdm(range(len(long_chunks))):
    content = json.dumps(
            long_chunks[i].page_content)
    long_chunks_embed[i] = np.array(embed(content))
    long_strings.append(content)

print(long_chunks_embed[0])

#long_embed = np.vstack(long_chunks_embed)

long_search.fit(long_chunks_embed)

joblib.dump(long_search, 'long_search_sk_nomic.pkl')


 11%|█████                                        | 140665/1246108 [19:47<2:40:06, 115.08it/s]

In [ ]:
with h5py.File("nomic_db.hdf5", "w") as f:
    long_set = f.create_dataset("long_chunks", (len(long_strings),), maxshape = (None,), dtype=h5py.string_dtype())
    long_set[:] = long_strings

In [ ]:
"""
with h5py.File("chunks_db.hdf5", "r") as f:
    print(str(f['long_chunks'][12500]))
"""

In [ ]:
"""
from cuml.neighbors import NearestNeighbors as cuNN
long_cusearch = cuNN(metric = 'cosine')
long_cusearch.fit(long_chunks_embed)
"""

In [ ]:
"""
l2_cusearch = cuNN(metric = 'l2')
l2_cusearch.fit(long_chunks_embed)
joblib.dump(l2_cusearch, 'cul2search.pkl')
"""

In [ ]:
#joblib.dump(long_cusearch, 'cusearch.pkl')

In [ ]:
joblib.dump(long_chunks_embed, 'long_nomic_embed.pkl')

In [ ]:
short_chunks = joblib.load('short_chunks.pkl')

short_chunks_embed = []

short_chunks_embed = np.zeros((len(short_chunks), 768))
short_strings =[]
for i in tqdm(range(len(long_chunks))):
    content = json.dumps(
            short_chunks[i].page_content)
    short_chunks_embed[i] = np.array(embed(content))
    short_strings.append(content)
    
for i in tqdm(short_chunks):
    short_chunks_embed.append(embed(
        json.dumps(
            i.page_content)))




short_search.fit(short_embed)

joblib.dump(short_search, 'short_search_sk_nomic.pkl')

with h5py.File("nomic_db.hdf5", "w") as f:
    short_set = f.create_dataset("short_chunks", (len(short_strings),), maxshape = (None,), dtype=h5py.string_dtype())
    short_set[:] = short_strings
